In [1]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy
import data_generation as dg
import time

In [7]:
#import import_ipynb
#we are importing the notebook files to avoid code redundancy
#python can do almost anything, except run fast
import s_metaheuristics as smt

In [8]:
import tabu_search as ts

In [10]:
graph, random_solution, total_edge_length = dg.read_instance('data\data8.txt')
num_nodes = len(graph)

In [11]:
def tabu_local_search_combo(graph, solution, max_iterations, neighbor_gen_func, local_search_func, change_func ,tabu_tenure):
    current_solution = deepcopy(solution)
    current_value = dg.calculate_total_edge_length(graph, current_solution)
    best_solution = current_solution.copy()
    best_value = current_value
    tabu_list = []

    best_i = -1

    for ind in range(max_iterations):
        current_solution, _, _ = deepcopy(local_search_func(graph, current_solution, current_value, int(max_iterations/20), change_func))

        neighbors = neighbor_gen_func(current_solution)
        neighbors.sort(key=lambda solution: dg.calculate_total_edge_length(graph, solution))
        #print(neighbors[0], calculate_total_edge_length(graph, neighbors[0]))

        
        next_solution = None
        for neighbor in neighbors:
            if neighbor not in tabu_list and dg.calculate_total_edge_length(graph, neighbor) <= current_value:
                next_solution = neighbor
                break


        if next_solution is None:
            next_solution = neighbors[0]


        tabu_list.append(next_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)


        current_solution = deepcopy(next_solution)
        current_value = dg.calculate_total_edge_length(graph, current_solution)

        # Update best solution
        if current_value < best_value:
            #print(current_value)
            best_i = ind
            #print('promena best_i', best_i)
            best_solution = deepcopy(current_solution)
            best_value = current_value

    return best_solution, best_value, best_i

In [12]:
#test
#solution, value, iter = tabu_local_search_combo(graph, random_solution, 500, ts.tabu_inverse, smt.local_search, smt.make_change_scramble ,5)
#print(solution, value, iter)

In [13]:
#let's itterate through the best combinations, instead of manually calling each combo and remembering each result
neighbor_generators = [ts.tabu_swap_neighbors, ts.tabu_inverse, ts.tabu_inverse_complete, ts.tabu_scramble, ts.tabu_scramble_continuous]
change_functions = [smt.make_change_swap, smt.make_change_scramble, smt.make_change_inverse]
search_functions = [smt.local_search, smt.simulated_annealing]

number_of_combinations = 0

sols = []
values =[]
times = []

methods = []

results = []

best = float('inf')
best_i = -1

average = [0,0]

for neighbor_gen in neighbor_generators:
    for search in search_functions:
        for change in change_functions:

            start = time.time()    
            sol, val, iter = tabu_local_search_combo(graph,
                                          random_solution,
                                          500,
                                          neighbor_gen,
                                          search,
                                          change,
                                          5)
            end = time.time()
            duration = float("{:.2f}".format(end - start))
            
            print(neighbor_gen.__name__, search.__name__, change.__name__, ' : ', sol, val, iter, '\n')

            sols.append(sols)
            values.append(val)
            times.append(duration)

            methods.append(str(neighbor_gen.__name__ + ', ' + search.__name__ + ', ' + change.__name__))

            results.append({'Dim': num_nodes, 'Method': methods[number_of_combinations], 'Value': abs(val), 'Time': duration})

            if val <= best:
                if duration <= times[best_i]:    
                    best = val
                    best_i = number_of_combinations

            average[0] += val
            average[1] += duration

            number_of_combinations += 1    



tabu_swap_neighbors local_search make_change_swap  :  [7, 4, 3, 2, 6, 0, 1, 5] 31 4 

tabu_swap_neighbors local_search make_change_scramble  :  [7, 3, 4, 2, 6, 0, 1, 5] 31 0 

tabu_swap_neighbors local_search make_change_inverse  :  [7, 4, 3, 2, 6, 0, 1, 5] 31 1 

tabu_swap_neighbors simulated_annealing make_change_swap  :  [7, 3, 4, 2, 6, 5, 1, 0] 31 3 

tabu_swap_neighbors simulated_annealing make_change_scramble  :  [7, 3, 4, 2, 6, 0, 1, 5] 31 1 

tabu_swap_neighbors simulated_annealing make_change_inverse  :  [7, 4, 3, 2, 6, 5, 0, 1] 31 2 

tabu_inverse local_search make_change_swap  :  [7, 4, 3, 2, 6, 5, 0, 1] 31 1 

tabu_inverse local_search make_change_scramble  :  [0, 1, 5, 6, 2, 4, 3, 7] 31 6 

tabu_inverse local_search make_change_inverse  :  [7, 3, 4, 2, 6, 0, 1, 5] 31 2 

tabu_inverse simulated_annealing make_change_swap  :  [7, 3, 4, 2, 6, 0, 5, 1] 31 3 

tabu_inverse simulated_annealing make_change_scramble  :  [7, 3, 4, 2, 6, 5, 1, 0] 31 9 

tabu_inverse simulated_anneal

In [14]:
df = pd.DataFrame(results)
display(df)
df.to_csv('comparison_tables/tabu_local_combo.csv', mode='a', header=not pd.io.common.file_exists('comparison_tables/tabu_local_combo.csv'), index=False)

,Dim,Method,Value,Time
0,8,"tabu_swap_neighbors, local_search, make_change...",31,0.39
1,8,"tabu_swap_neighbors, local_search, make_change...",31,0.52
2,8,"tabu_swap_neighbors, local_search, make_change...",31,0.36
3,8,"tabu_swap_neighbors, simulated_annealing, make...",31,0.38
4,8,"tabu_swap_neighbors, simulated_annealing, make...",31,0.56
5,8,"tabu_swap_neighbors, simulated_annealing, make...",31,0.40
6,8,"tabu_inverse, local_search, make_change_swap",31,0.35
7,8,"tabu_inverse, local_search, make_change_scramble",31,0.54
8,8,"tabu_inverse, local_search, make_change_inverse",31,0.38
9,8,"tabu_inverse, simulated_annealing, make_change...",31,0.42


In [15]:
print('best:', methods[best_i], values[best_i], times[best_i])
df_best = pd.DataFrame({'Dim': num_nodes, 'Method':  methods[best_i], 'Value': values[best_i], 'Time': times[best_i]}, index=[0])
display(df_best)

best: tabu_inverse, local_search, make_change_swap 31 0.35


,Dim,Method,Value,Time
0,8,"tabu_inverse, local_search, make_change_swap",31,0.35


In [16]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'taloc_combo'] = values[best_i]
    df.loc[row_to_update.index, 'taloc_combo_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'taloc_combo': values[best_i], 'taloc_combo_time': times[best_i]}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [17]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': num_nodes, 'Value': average[0], 'Time': average[1]}, index=[0])
display(df_avg)

average: [31.0, 0.6]


,Dim,Value,Time
0,8,31.0,0.6


In [18]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'taloc_combo'] = average[0]
    df.loc[row_to_update.index, 'taloc_combo_time'] = average[1]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'taloc_combo': average[0], 'taloc_combo_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)